<a href="https://colab.research.google.com/github/vash6618/distributed_systems_project/blob/main/HAR_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas import read_csv
from numpy import dstack
import tensorflow as tf
from tensorflow.keras.utils import to_categorical 

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [ ]:

# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [ ]:
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [ ]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainX, trainy, testX, testy = load_dataset('/content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/Research project/')

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.layers import ConvLSTM2D 
import numpy as np

In [ ]:
# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = np.nanmean(scores), np.nanstd(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
def evaluate_conv_lstm_model(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 25, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape into subsequences (samples, time steps, rows, cols, channels)
	n_steps, n_length = 4, 32
	trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
	# define model
	model = Sequential()
	model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy, model

In [ ]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy, model

In [ ]:
def run_experiment_with_conv_lstm(repeats=1):
  # trainX, trainy, testX, testy = load_dataset()
  scores = list()
  models = list()
  for r in range(repeats):
    score, model = evaluate_conv_lstm_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
    models.append(model)
  summarize_results(scores)
  return models

In [ ]:
def run_experiment_with_lstm(repeats=1):
  # trainX, trainy, testX, testy = load_dataset()
  scores = list()
  models = list()
  for r in range(repeats):
    score, model = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
    models.append(model)
  summarize_results(scores)
  return models

In [ ]:
models = run_experiment_with_lstm()

>#1: 90.499
[90.49881100654602]
Accuracy: 90.499% (+/-0.000)


In [ ]:
models = run_experiment_with_conv_lstm()

In [ ]:
print(models)

In [ ]:
for model in models:
  print(model.layers[0])

In [ ]:
models[0].save('/content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/all_models/LSTM_normal_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/all_models/LSTM_normal_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/all_models/LSTM_normal_model/assets


In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/all_models/LSTM_normal_model')

In [ ]:
model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               44000     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 606       
Total params: 54,706
Trainable params: 54,706
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def print_model_sparsity(pruned_model):
  """Prints sparsity for the pruned layers in the model.
  Model Sparsity Summary
  --
  prune_lstm_1: (kernel, 0.5), (recurrent_kernel, 0.6)
  prune_dense_1: (kernel, 0.5)
  Args:
    pruned_model: keras model to summarize.
  Returns:
    None
  """
  def _get_sparsity(weights):
    return 1.0 - np.count_nonzero(weights) / float(weights.size)

  print("Model Sparsity Summary ({})".format(pruned_model.name))
  print("--")
  for layer in pruned_model.layers:
    if isinstance(layer, pruning_wrapper.PruneLowMagnitude):
      prunable_weights = layer.layer.get_prunable_weights()
      if prunable_weights:
        print("start prunable ----------------------------")
        print("{}: {}".format(
            layer.name, ", ".join([
                "({}, {})".format(weight.name,
                                  str(_get_sparsity(K.get_value(weight))))
                for weight in prunable_weights
            ])))
        print("stop prunable ----------------------------")
  print("\n")

In [ ]:
!pip install tensorflow_model_optimization
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import tensorflow.keras.preprocessing.sequence as sequence
from tensorflow_model_optimization.python.core.sparsity.keras import prune
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_schedule
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper

compressed_pruned_model = prune.prune_low_magnitude(model, pruning_schedule.PolynomialDecay(
    initial_sparsity=0.3, final_sparsity=0.7, begin_step=1000, end_step=3000))

print(compressed_pruned_model.summary())

# try using different optimizers and different optimizer configs
compressed_pruned_model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
# print_model_sparsity(pruned_model)
batch_size = 64


# print("Train...")
# pruned_model.fit(trainX, trainy, batch_size=batch_size, epochs=2,
#           callbacks=[pruning_callbacks.UpdatePruningStep()], validation_split=0.1)
score, acc = compressed_pruned_model.evaluate(testX, testy,
                            batch_size=batch_size)
# print_model_sparsity(pruned_model)
print("Test score:", score)
print("Test accuracy:", acc)

     |████████████████████████████████| 174kB 8.8MB/s 


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_lstm (Pr (None, 100)               87603     
_________________________________________________________________
prune_low_magnitude_dropout  (None, 100)               1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 100)               20102     
_________________________________________________________________
prune_low_magnitude_dense_1  (None, 6)                 1208      
Total params: 108,914
Trainable params: 54,706
Non-trainable params: 54,208
_________________________________________________________________
None
47/47 [==============================] - 3s 42ms/step - loss: 0.3706 - accuracy: 0.8626
Test score: 0.31666630506515503
Test accuracy: 0.9049881100654602


In [ ]:
# from tensorflow import keras

# compressed_pruned_model.save('/content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/all_models/LSTM_pruned_model')

# pruned_model_lstm = keras.models.load_model('/content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/LSTM_pruned_model')
# pruned_model_lstm.compile(loss="categorical_crossentropy",
#               optimizer="adam",
#               metrics=["accuracy"])

In [ ]:
import tensorflow_model_optimization as tfmot
export_pruned_LSTM = tfmot.sparsity.keras.strip_pruning(compressed_pruned_model)
export_pruned_LSTM.save('/content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/all_models/LSTM_pruned_exported')

# _, pruned_lstm_file = tempfile.mkstemp('tfpruned.h5')
# tf.keras.models.save_model(model_for_export, pruned_lstm_file, include_optimizer=False)
# print('Saved pruned Keras model to:', pruned_lstm_file)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/all_models/LSTM_pruned_exported/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/all_models/LSTM_pruned_exported/assets


In [ ]:
batch_size = 64


export_pruned_LSTM.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
# print("Train...")
# pruned_model.fit(trainX, trainy, batch_size=batch_size, epochs=2,
#           callbacks=[pruning_callbacks.UpdatePruningStep()], validation_split=0.1)
score, acc = export_pruned_LSTM.evaluate(testX, testy,
                            batch_size=batch_size)
# print_model_sparsity(pruned_model)
print("Test score:", score)
print("Test accuracy:", acc)

47/47 [==============================] - 3s 41ms/step - loss: 0.3706 - accuracy: 0.8626
Test score: 0.31666630506515503
Test accuracy: 0.9049881100654602


In [ ]:
import tempfile

converter = tf.lite.TFLiteConverter.from_keras_model(export_pruned_LSTM)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('_quantized.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

# print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
# print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmpvn4z7z1g/assets


INFO:tensorflow:Assets written to: /tmp/tmpvn4z7z1g/assets


Saved quantized and pruned TFLite model to: /tmp/tmpdc58dx0y_quantized.tflite


In [ ]:
tflite_interpreter = tf.lite.Interpreter(model_path=quantized_and_pruned_tflite_file)

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()
print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: serving_default_lstm_input:0
shape: [  1 128   9]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [1 6]
type: <class 'numpy.float32'>


In [ ]:
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (1, 128, 9))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (1, 6))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("index:", input_details[0]['index'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: serving_default_lstm_input:0
shape: [  1 128   9]
type: <class 'numpy.float32'>
index: 0

== Output details ==
name: StatefulPartitionedCall:0
shape: [1 6]
type: <class 'numpy.float32'>


In [ ]:

x_test = tf.dtypes.cast(testX,tf.float32)
# x_test = x_test.numpy()
x_test[0:64, :, :]

<tf.Tensor: shape=(64, 128, 9), dtype=float32, numpy=
array([[[ 1.041216e+00, -2.697959e-01,  2.377977e-02, ...,
          4.374637e-01,  5.313492e-01,  1.365279e-01],
        [ 1.041803e+00, -2.800250e-01,  7.629271e-02, ...,
          4.682641e-01,  7.210685e-01,  9.762239e-02],
        [ 1.039086e+00, -2.926631e-01,  1.474754e-01, ...,
          4.982574e-01,  5.203284e-01,  8.355578e-02],
        ...,
        [ 9.930164e-01, -2.599865e-01,  1.443951e-01, ...,
         -5.055856e-03, -7.734212e-02,  3.225787e-02],
        [ 9.932414e-01, -2.620643e-01,  1.447033e-01, ...,
         -2.043194e-02, -7.297300e-02,  2.700848e-02],
        [ 9.943906e-01, -2.641348e-01,  1.454939e-01, ...,
         -2.999741e-02, -7.064875e-02,  3.054609e-02]],

       [[ 9.991921e-01, -2.649349e-01,  1.256164e-01, ...,
          1.702878e-01, -6.137388e-02,  5.509624e-02],
        [ 9.946787e-01, -2.532142e-01,  1.256249e-01, ...,
          1.752221e-01, -9.536355e-02,  4.334361e-02],
        [ 9.935518e

In [ ]:
output_list = []
for i in range(2947):
  tflite_interpreter.set_tensor(input_details[0]['index'], x_test[i:i+1, :, :])

  tflite_interpreter.invoke()

  tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
  output_list += tflite_model_predictions.tolist()
# print("Prediction results shape:", tflite_model_predictions.shape)
# print("Actual output :- ", tflite_model_predictions)
print(len(output_list))
print(output_list[0:10])

2947
[[0.00025476032169535756, 4.884186637355015e-05, 0.0002797898487187922, 0.05096079036593437, 0.948441207408905, 1.455172423447948e-05], [0.00014442350948229432, 2.3915972633403726e-05, 0.00015379564138129354, 0.039592936635017395, 0.960078239440918, 6.6443808464100584e-06], [0.0001280668657273054, 2.0475148630794138e-05, 0.00013545829278882593, 0.03586583957076073, 0.963844895362854, 5.31522437086096e-06], [0.00010479502816451713, 1.658186738495715e-05, 0.00010883259528782219, 0.03468989580869675, 0.9650756120681763, 4.305918992031366e-06], [0.00010183606354985386, 1.662605609453749e-05, 0.00010567524441285059, 0.03524995595216751, 0.9645214676856995, 4.534772415354382e-06], [0.00010667312744772062, 1.7345226297038607e-05, 0.00011127660400234163, 0.034875087440013885, 0.9648849964141846, 4.54373821412446e-06], [0.00010882057540584356, 1.725985566736199e-05, 0.00011263418855378404, 0.03281492739915848, 0.9669422507286072, 4.202504896966275e-06], [0.00010193466732744128, 1.614008397

In [ ]:
print(type(testy))

<class 'numpy.ndarray'>


In [ ]:
sum_tot = 0
for i in range(len(output_list)):
  ind = np.argmax(output_list[i])
  if testy[i][ind] == 1:
    sum_tot += 1
print("accuracy :- ", sum_tot/len(output_list))

accuracy :-  0.9046487953851374


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model)
model_for_export.save('/content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/exported_model')
# _, model_file = tempfile.mkstemp('tforiginal.h5')
# tf.keras.models.save_model(model_for_export, model_file, include_optimizer=False)
# print('Saved Keras model to:', model_file)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/exported_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/CU Boulder Classes/Distributed Systems CU Boulder/exported_model/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp8wl4vbd3/assets


INFO:tensorflow:Assets written to: /tmp/tmp8wl4vbd3/assets


In [ ]:
# ---- applying weight clustering and quantization from this point
export_pruned_LSTM.summary()